In [30]:
from audioop import avg
import datetime

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F


In [31]:
conf = SparkConf()
conf.setMaster("local").setAppName("My app")

sc = SparkContext.getOrCreate(conf=conf)
spark_session = SparkSession(sc)

In [32]:
df_1 = spark_session.createDataFrame(
    data=[
        [datetime.date(2021, 1, 1), 'demo', 1.123, 10],
        [datetime.date(2021, 2, 2), 'demo', 5.286, 10]
    ],
    schema=StructType(
        [
            StructField('date', DateType(), True),
            StructField('type', StringType(), True),
            StructField('value', DoubleType(), True),
            StructField('count', LongType(), True),
        ]
    ),
)

df_2 = spark_session.createDataFrame(
    data=[
        [datetime.date(2020, 1, 1), 'demo', 4.321, 10],
        [datetime.date(2023, 3, 3), 'demo', 6.900, 10]
    ],
    schema=StructType(
        [
            StructField('date', DateType(), True),
            StructField('type', StringType(), True),
            StructField('value', DoubleType(), True),
            StructField('count', LongType(), True),
        ]
    ),
)

In [33]:
df_1.show()
df_2.show()

+----------+----+-----+-----+
|      date|type|value|count|
+----------+----+-----+-----+
|2021-01-01|demo|1.123|   10|
|2021-02-02|demo|5.286|   10|
+----------+----+-----+-----+

+----------+----+-----+-----+
|      date|type|value|count|
+----------+----+-----+-----+
|2020-01-01|demo|4.321|   10|
|2023-03-03|demo|  6.9|   10|
+----------+----+-----+-----+



In [34]:
df_3 = df_1.join(df_2)
df_3.show()

+----------+----+-----+-----+----------+----+-----+-----+
|      date|type|value|count|      date|type|value|count|
+----------+----+-----+-----+----------+----+-----+-----+
|2021-01-01|demo|1.123|   10|2020-01-01|demo|4.321|   10|
|2021-01-01|demo|1.123|   10|2023-03-03|demo|  6.9|   10|
|2021-02-02|demo|5.286|   10|2020-01-01|demo|4.321|   10|
|2021-02-02|demo|5.286|   10|2023-03-03|demo|  6.9|   10|
+----------+----+-----+-----+----------+----+-----+-----+



In [35]:
df_4 = df_1.union(df_2)
df_4.show()

+----------+----+-----+-----+
|      date|type|value|count|
+----------+----+-----+-----+
|2021-01-01|demo|1.123|   10|
|2021-02-02|demo|5.286|   10|
|2020-01-01|demo|4.321|   10|
|2023-03-03|demo|  6.9|   10|
+----------+----+-----+-----+



In [37]:
df_5 = df_4.groupBy("type").agg(F.min("value"),F.max("value"),F.avg("value"),F.sum("value")) # {"col_c":"max"}, {"col_c":"avg"}, {"col_c":"sum"}
df_5.show()

+----+----------+----------+----------+----------+
|type|min(value)|max(value)|avg(value)|sum(value)|
+----+----------+----------+----------+----------+
|demo|     1.123|       6.9|    4.4075|     17.63|
+----+----------+----------+----------+----------+



In [ ]:
df_6 = df_1.union(df_2) \
        .select(df_1.value)
df_6.show()

+-----+
|col_c|
+-----+
|1.123|
|5.286|
|4.321|
|  6.9|
+-----+



In [ ]:
df_7 = df_1.union(df_2) \
        .select(df_1.date, df_1.value) \
        .filter(df_1.date > datetime.date(2020, 2, 3))
df_7.show()

+----------+-----+
|     col_a|col_c|
+----------+-----+
|2021-01-01|1.123|
|2021-02-02|5.286|
|2023-03-03|  6.9|
+----------+-----+



In [48]:
df_4.show()

df_4.foreach(lambda f: print(f))

print('Hi')


+----------+----+-----+-----+
|      date|type|value|count|
+----------+----+-----+-----+
|2021-01-01|demo|1.123|   10|
|2021-02-02|demo|5.286|   10|
|2020-01-01|demo|4.321|   10|
|2023-03-03|demo|  6.9|   10|
+----------+----+-----+-----+

Hi


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51295)
Traceback (most recent call last):
  File "c:\Users\Elizabeth\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\Elizabeth\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\Elizabeth\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\Elizabeth\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "c:\Users\Elizabeth\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\accumulators.py", line 281, in handle
    poll(accum_updates)
  File "c:\Users\Elizabeth\AppData\Local\Programs\P